Copy rerepsentative datasets from google drive (a zipped folder with 200~500 images)

In [ ]:
!cp /content/drive/MyDrive/chosen.zip /content
!unzip /content/chosen.zip -d /content/

Install required packages

In [ ]:
!pip install ultralytics

In [ ]:
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | tee /etc/apt/sources.list.d/coral-edgetpu.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt-get update && apt-get install -y edgetpu-compiler

Load the model and export to Tensorflow SavedModel

In [ ]:
from ultralytics import YOLO
model = YOLO("/content/yolov8n_300e.pt")

model.export(format="saved_model", imgsz=320, nms=True, conf=0.6, iou=0.4, agnostic_nms=False, max_det=100)

Get the quantization script and run the script

In [ ]:
!wget https://raw.githubusercontent.com/selfhosteverything/frigate-yolo-edgetpu/refs/heads/main/convert_to_tflite_uint8.py -O /content/convert_to_tflite_uint8.py

In [ ]:
!python3 convert_to_tflite_uint8.py /content/yolov8n_300e_saved_model/ /content/chosen/ /content/yolov8n_300e.tflite --img_height=320 --img_width=320

Compile for edgetpu

In [ ]:
!edgetpu_compiler /content/yolov8n_300e.tflite -s

Clean up

In [ ]:
!rm -r uint8_models/
!rm -r *_saved_model/
!rm *.onnx
!rm *.log
!rm calibration*